In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
from collections import Counter
warnings.filterwarnings('ignore')
%matplotlib inline

In [3]:
import eli5

---

In [3]:
#!wget https://github.com/buriy/russian-nlp-datasets/releases/download/r4/lenta.tar.bz2

In [4]:
data = pd.read_csv('news_lenta.csv.gz')
print(data.shape)
data.head()

(699746, 5)


,tags,text,title,topic,url
0,Общество,Миллиардер Илон Маск в резкой форме ответил бр...,Илон Маск назвал педофилом спасавшего детей из...,Мир,https://lenta.ru/news/2018/07/16/su57/
1,Рынки,США и их западные союзники рассматривают возмо...,США задумались о распечатывании нефтяного резерва,Экономика,https://lenta.ru/news/2018/07/16/foes/
2,Преступность,Празднование победы сборной Франции на чемпион...,Празднование победы на ЧМ во Франции закончило...,Мир,https://lenta.ru/news/2018/07/15/fra_bezumie/
3,Политика,География использования лимузинов проекта «Кор...,Песков рассказал о планах на президентские лим...,Россия,https://lenta.ru/news/2018/07/16/delo_shyut/
4,Музыка,Американская поп-певица Бритни Спирс случайно ...,Грудь Бритни Спирс вновь выскочила из лифчика ...,Культура,https://lenta.ru/news/2018/07/16/pedomusk/


In [5]:
data = pd.read_csv('news_lenta.csv.gz')
print(data.shape)
data.head()

(699746, 5)


,tags,text,title,topic,url
0,Общество,Миллиардер Илон Маск в резкой форме ответил бр...,Илон Маск назвал педофилом спасавшего детей из...,Мир,https://lenta.ru/news/2018/07/16/su57/
1,Рынки,США и их западные союзники рассматривают возмо...,США задумались о распечатывании нефтяного резерва,Экономика,https://lenta.ru/news/2018/07/16/foes/
2,Преступность,Празднование победы сборной Франции на чемпион...,Празднование победы на ЧМ во Франции закончило...,Мир,https://lenta.ru/news/2018/07/15/fra_bezumie/
3,Политика,География использования лимузинов проекта «Кор...,Песков рассказал о планах на президентские лим...,Россия,https://lenta.ru/news/2018/07/16/delo_shyut/
4,Музыка,Американская поп-певица Бритни Спирс случайно ...,Грудь Бритни Спирс вновь выскочила из лифчика ...,Культура,https://lenta.ru/news/2018/07/16/pedomusk/


In [3]:
df = pd.read_csv('lenta.tar.bz2',usecols = ['topics','text'])
print(df.shape)
df.head()

(700006, 2)


,topics,text
0,Библиотека / Первая мировая,Бои у Сопоцкина и Друскеник закончились отступ...
1,Библиотека / Первая мировая,"Министерство народного просвещения, в виду про..."
2,Библиотека / Первая мировая,Фотограф-корреспондент Daily Mirror рассказыва...
3,Библиотека / Первая мировая,"Штабс-капитан П. Н. Нестеров на днях, увидев в..."
4,Библиотека / Первая мировая,"Лица, приехавшие в Варшаву из Люблина, передаю..."


In [4]:
df.dropna(inplace=True)
df.isnull().sum()

topics    0
text      0
dtype: int64

In [5]:
df.topics.unique().shape[0]

125

In [6]:
df.topics.value_counts().head()

Россия       116891
Мир           95099
Экономика     58247
Спорт         42323
Культура      34609
Name: topics, dtype: int64

In [7]:
take1st5topics = df.topics.value_counts().head(5).index
take1st5topics

Index(['Россия', 'Мир', 'Экономика', 'Спорт', 'Культура'], dtype='object')

In [8]:
df = df[df.topics.isin(take1st5topics)]
df.shape

(347169, 2)

---

In [9]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [10]:
le.fit(df.topics)
list(le.classes_)

['Культура', 'Мир', 'Россия', 'Спорт', 'Экономика']

In [11]:
le.transform(df.topics) 

array([2, 2, 2, ..., 4, 3, 3])

In [12]:
le.transform(['Россия'])[0]

2

In [13]:
df['target'] = le.transform(df.topics)
df.drop(['topics'],axis=1,inplace=True)
df.reset_index(inplace=True,drop=True)

df.sample(5)

,text,target
12507,"Актриса Дэрил Ханна (""Бегущий по лезвию бритвы...",0
146133,В пятницу в порту Анадыря опрокинулась самоход...,1
189293,В городе Акко на севере Израиля 10 октября про...,1
218045,"Группировка ""Радикальный народный коммунистиче...",3
102264,Сотрудники МВД на прошлой неделе взяли под сво...,2


---

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['target'],axis=1),df.target,
                                                    test_size=0.2,
                                                    stratify=df.target,
                                                    random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((277735, 1), (69434, 1), (277735,), (69434,))

In [16]:
X_test.isna().sum()

text    0
dtype: int64

____

In [17]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression 

import itertools
from itertools import islice

### построим простую модельку без препроцессинга

In [18]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

* without `min_df`

In [19]:
%%time
cv_notPrep_notMindf = CountVectorizer()
cv_notPrep_notMindf_train = cv_notPrep_notMindf.fit_transform(X_train.text)
cv_notPrep_notMindf_test = cv_notPrep_notMindf.transform(X_test.text)

CPU times: user 43.1 s, sys: 819 ms, total: 43.9 s
Wall time: 43.9 s


In [20]:
dict_bwnotPrep_notMindf = cv_notPrep_notMindf.vocabulary_

In [21]:
len(dict_bwnotPrep_notMindf)

629073

In [64]:
#for key, value in sorted(dict_bwnotPrep_notMindf.items(), key=lambda item: item[1]):
 #   print(f'{key}: {value}')

In [22]:
dict_bwnotPrep_notMindf_sorted = sorted(dict_bwnotPrep_notMindf.items(), key=lambda item: item[1])
len(dict_bwnotPrep_notMindf_sorted)

629073

In [23]:
dict_bwnotPrep_notMindf_sorted[:10]

[('00', 0),
 ('000', 1),
 ('0000', 2),
 ('00000000', 3),
 ('00000000301', 4),
 ('0000000498308148918584', 5),
 ('0000001', 6),
 ('0000004', 7),
 ('0000005', 8),
 ('0000007', 9)]

In [24]:
dict_bwnotPrep_notMindf_sorted[-10:]

[('ј60m', 629063),
 ('ј65m', 629064),
 ('ј8', 629065),
 ('ј8m', 629066),
 ('ј9m', 629067),
 ('љубав', 629068),
 ('ジェーニャマトリョーシカ', 629069),
 ('市内のお土産屋さんがわざわざ出してくれました', 629070),
 ('微妙', 629071),
 ('番外編', 629072)]

In [25]:
%%time
lr = LogisticRegression()
lr.fit(cv_notPrep_notMindf_train,y_train)

pred_lr = lr.predict(cv_notPrep_notMindf_test)
print('accuracy =',accuracy_score(y_test,pred_lr))
print('\nclassification_report:')
print(classification_report(y_test,pred_lr))
print('\nconfusion_matrix:')
print(confusion_matrix(y_test,pred_lr))

accuracy = 0.9114410807385431

classification_report:
              precision    recall  f1-score   support

           0       0.94      0.94      0.94      6922
           1       0.88      0.88      0.88     19020
           2       0.89      0.90      0.89     23378
           3       0.98      0.98      0.98      8465
           4       0.93      0.93      0.93     11649

   micro avg       0.91      0.91      0.91     69434
   macro avg       0.93      0.92      0.93     69434
weighted avg       0.91      0.91      0.91     69434


confusion_matrix:
[[ 6488   209   187    11    27]
 [  195 16776  1753    48   248]
 [  168  1760 20928    56   466]
 [   14    55    88  8290    18]
 [   24   278   519    25 10803]]
CPU times: user 14min 6s, sys: 215 ms, total: 14min 6s
Wall time: 14min 6s


___

* with `min_df`

In [ ]:
%%time
cv_notPrep_withMindf = CountVectorizer(min_df=0.1)
cv_notPrep_withMindf_train = cv_notPrep_withMindf.fit_transform(X_train.text)
cv_notPrep_withMindf_test = cv_notPrep_withMindf.transform(X_test.text)

In [ ]:
dict_bw_notPrep_withMindf = cv_notPrep_withMindf.vocabulary_

In [ ]:
len(dict_bw_notPrep_withMindf)

In [ ]:
dict_bw_notPrep_withMindf_sorted = sorted(dict_bw_notPrep_withMindf.items(), key=lambda item: item[1])
len(dict_bw_notPrep_withMindf_sorted)

In [ ]:
dict_bw_notPrep_withMindf_sorted[:10]

In [ ]:
dict_bw_notPrep_withMindf_sorted[-10:]

In [75]:
%%time
lr = LogisticRegression()
lr.fit(cv_notPrep_withMindf_train,y_train)

pred_lr = lr.predict(cv_notPrep_withMindf_test)
print('accuracy =',accuracy_score(y_test,pred_lr))
print('\nclassification_report:')
print(classification_report(y_test,pred_lr))
print('\nconfusion_matrix:')
print(confusion_matrix(y_test,pred_lr))

accuracy = 0.7625999111900533

classification_report:
              precision    recall  f1-score   support

           0       0.74      0.80      0.77     19020
           1       0.77      0.78      0.77     23378
           2       0.79      0.68      0.73     11650

   micro avg       0.76      0.76      0.76     54048
   macro avg       0.77      0.75      0.76     54048
weighted avg       0.76      0.76      0.76     54048


confusion_matrix:
[[15175  3146   699]
 [ 3913 18120  1345]
 [ 1448  2280  7922]]
CPU times: user 25 s, sys: 7.78 ms, total: 25 s
Wall time: 25 s


In [ ]:
#pipe = Pipeline([('vec', CountVectorizer(min_df=0.1)),
#                 ('clf', LogisticRegression())
#                 ]
#               )
#                  
#pipe.fit(X_train,y_train)
#
#pred_lr = pipe.predict(X_test)
#print('accuracy =',accuracy_score(y_test,pred_lr))
#print('\nclassification_report:')
#print(classification_report(y_test,pred_lr))
#print('\nconfusion_matrix:')
#print(confusion_matrix(y_test,pred_lr))

### построим простую модельку с препроцессингом

In [33]:
import re
from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize

from razdel import tokenize # https://github.com/natasha/razdel
#!pip install razdel

import pymorphy2  # pip install pymorphy2

In [34]:
morph = pymorphy2.MorphAnalyzer()

In [35]:
stopword_ru = stopwords.words('russian')
len(stopword_ru)

with open('../stopwords.txt', 'r', encoding='utf-8') as f:
    for w in f.readlines():
        stopword_ru.append(w)
        
len(stopword_ru)

775

* clean_text

In [36]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
   
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')

    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())

    return text

In [38]:
df.head(2)

,text,target
0,В минувший четверг президент РФ Борис Ельцин п...,1
1,ИТАР-ТАСС со ссылкой на пресс-службу Миноборон...,1


In [37]:
%%time
X_train['clean_text'] = X_train.text.apply(clean_text)
X_test['clean_text'] = X_test.text.apply(clean_text)

CPU times: user 1min 36s, sys: 184 ms, total: 1min 36s
Wall time: 1min 36s


* lemmatization

In [38]:
cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка на стоп-слова
        [4] проверка токена с одного символа
        [5] проверка есть ли данное слово в кэше
        [6] лемматизация слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    
    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if not w in stopword_ru: # [3]
            if len(w)>1: # [4]
                if w in cache: # [5]
                    words_lem.append(cache[w])
                else: # [6]
                    temp_cach = cache[w] = morph.parse(w)[0].normal_form
                    words_lem.append(temp_cach)
    return words_lem

In [39]:
%%time
X_train['lemma_clean_text'] = X_train.clean_text.apply(lemmatization)
X_test['lemma_clean_text'] = X_test.clean_text.apply(lemmatization)

CPU times: user 11min 58s, sys: 508 ms, total: 11min 58s
Wall time: 11min 58s


In [89]:
X_train['join_lemma_clean_text'] = X_train.lemma_clean_text.apply(lambda x: ' '.join(x))
X_train.head(2)

,text,clean_text,lemma_clean_text,join_lemma_clean_text
152792,В начале апреля завершилась процедура предъявл...,в начале апреля завершилась процедура предъявл...,"[начало, апрель, завершиться, процедура, предъ...",начало апрель завершиться процедура предъявлен...
183485,"Уполномоченная правительства ФРГ по архивам ""Ш...","уполномоченная правительства фрг по архивам ""ш...","[уполномоченный, правительство, фрг, архив, шт...",уполномоченный правительство фрг архив штази м...


In [90]:
X_test['join_lemma_clean_text'] = X_test.lemma_clean_text.apply(lambda x: ' '.join(x))
X_test.head(2)

,text,clean_text,lemma_clean_text,join_lemma_clean_text
183168,На предстоящих 11 октября выборах в Мосгордуму...,на предстоящих октября выборах в мосгордуму в...,"[предстоящий, октябрь, выборы, мосгордума, все...",предстоящий октябрь выборы мосгордума всеросси...
153073,Высокий курс евро по отношению к доллару приве...,высокий курс евро по отношению к доллару приве...,"[высокий, курс, евро, отношение, доллар, приве...",высокий курс евро отношение доллар привести ув...


* without `min_df`

In [91]:
%%time
cv_withPrep_notMindf = CountVectorizer()
cv_withPrep_notMindf_train = cv_withPrep_notMindf.fit_transform(X_train.join_lemma_clean_text)
cv_withPrep_notMindf_test = cv_withPrep_notMindf.transform(X_test.join_lemma_clean_text)

CPU times: user 23.1 s, sys: 172 ms, total: 23.2 s
Wall time: 23.2 s


In [92]:
dict_bw_withPrep_notMindf = cv_withPrep_notMindf.vocabulary_

In [93]:
len(dict_bw_withPrep_notMindf)

272274

In [ ]:
#for key, value in sorted(dict_bwnotPrep_notMindf.items(), key=lambda item: item[1]):
 #   print(f'{key}: {value}')

In [94]:
dict_bw_withPrep_notMindf_sorted = sorted(dict_bw_withPrep_notMindf.items(), key=lambda item: item[1])
len(dict_bw_withPrep_notMindf_sorted)

272274

In [95]:
dict_bw_withPrep_notMindf_sorted[:10]

[('aa', 0),
 ('aaa', 1),
 ('aaaa', 2),
 ('aaae', 3),
 ('aaaru', 4),
 ('aaarus', 5),
 ('aabar', 6),
 ('aabid', 7),
 ('aabp', 8),
 ('aac', 9)]

In [96]:
dict_bw_withPrep_notMindf_sorted[-10:]

[('ёон', 272264),
 ('ёпартия', 272265),
 ('ёрико', 272266),
 ('ёрничать', 272267),
 ('ёрш', 272268),
 ('ёсихико', 272269),
 ('ёёга', 272270),
 ('єр', 272271),
 ('іnternatіonal', 272272),
 ('іі', 272273)]

In [97]:
%%time
lr = LogisticRegression()
lr.fit(cv_withPrep_notMindf_train,y_train)

pred_lr = lr.predict(cv_withPrep_notMindf_test)
print('accuracy =',accuracy_score(y_test,pred_lr)) # without prepr accuracy = 0.9053989046773239
print('\nclassification_report:')
print(classification_report(y_test,pred_lr))
print('\nconfusion_matrix:')
print(confusion_matrix(y_test,pred_lr))

accuracy = 0.9046773238602723

classification_report:
              precision    recall  f1-score   support

           0       0.89      0.89      0.89     19020
           1       0.90      0.90      0.90     23378
           2       0.94      0.93      0.93     11650

   micro avg       0.90      0.90      0.90     54048
   macro avg       0.91      0.91      0.91     54048
weighted avg       0.90      0.90      0.90     54048


confusion_matrix:
[[17012  1736   272]
 [ 1806 21096   476]
 [  319   543 10788]]
CPU times: user 7min 31s, sys: 69 ms, total: 7min 31s
Wall time: 7min 31s


* with `min_df`

In [113]:
%%time
cv_withPrep_withMindf = CountVectorizer(min_df=0.1)
cv_withPrep_withMindf_train = cv_withPrep_withMindf.fit_transform(X_train.join_lemma_clean_text)
cv_withPrep_withMind_test = cv_withPrep_withMindf.transform(X_test.join_lemma_clean_text)

CPU times: user 21.6 s, sys: 124 ms, total: 21.8 s
Wall time: 21.8 s


In [114]:
dict_bw_withPrep_withMindf = cv_withPrep_withMindf.vocabulary_

In [115]:
len(dict_bw_withPrep_withMindf)

111

In [116]:
#for key, value in sorted(dict_bwnotPrep_notMindf.items(), key=lambda item: item[1]):
 #   print(f'{key}: {value}')

In [117]:
dict_bw_withPrep_withMindf_sorted = sorted(dict_bw_withPrep_withMindf.items(), key=lambda item: item[1])
len(dict_bw_withPrep_withMindf_sorted)

111

In [118]:
dict_bw_withPrep_withMindf_sorted[:10]

[('агентство', 0),
 ('акция', 1),
 ('американский', 2),
 ('бывший', 3),
 ('быть', 4),
 ('власть', 5),
 ('военный', 6),
 ('вопрос', 7),
 ('время', 8),
 ('газета', 9)]

In [119]:
dict_bw_withPrep_withMindf_sorted[-10:]

[('тысяча', 101),
 ('управление', 102),
 ('ход', 103),
 ('частность', 104),
 ('часть', 105),
 ('человек', 106),
 ('число', 107),
 ('член', 108),
 ('это', 109),
 ('являться', 110)]

In [120]:
%%time
lr = LogisticRegression()
lr.fit(cv_withPrep_withMindf_train,y_train)

pred_lr = lr.predict(cv_withPrep_withMind_test)
print('accuracy =',accuracy_score(y_test,pred_lr)) # without prepr accuracy = 0.7625999111900533
print('\nclassification_report:')
print(classification_report(y_test,pred_lr))
print('\nconfusion_matrix:')
print(confusion_matrix(y_test,pred_lr))

accuracy = 0.8064498223801065

classification_report:
              precision    recall  f1-score   support

           0       0.78      0.82      0.80     19020
           1       0.81      0.81      0.81     23378
           2       0.85      0.76      0.80     11650

   micro avg       0.81      0.81      0.81     54048
   macro avg       0.81      0.80      0.81     54048
weighted avg       0.81      0.81      0.81     54048


confusion_matrix:
[[15690  2732   598]
 [ 3421 19013   944]
 [ 1081  1685  8884]]
CPU times: user 14 s, sys: 0 ns, total: 14 s
Wall time: 14 s
